In [25]:
import numpy as np
import pandas as pd
import tensorflow as tf

#learning
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models
from tensorflow.keras.utils import to_categorical

# different models
from tensorflow.keras.applications import efficientnet_v2




#data augmentation
from tensorflow.keras.preprocessing.image import ImageDataGenerator

#preprocessing images
import cv2

## Preprocessing and loading in the data

In [26]:
# the values per image that belong to the given attributes for each class
attribute_values = np.load('feather-in-focus/attributes.npy')

# names of each bird class
labels = np.load("feather-in-focus/class_names.npy", allow_pickle=True).item()

# list of training data containing: image location, labels per image 
data = pd.read_csv("feather-in-focus/train_images.csv")


In [27]:
# code that adds the attributes per label to the training_df
att_list = []
for i in range(len(data)):
    att_list.append(attribute_values[data['label'][i]-1])  
    
data['attributes'] = att_list

In [28]:
def load_and_preprocess_images(image_paths, file_path):
    images = []
    for img_path in image_paths:
        img = cv2.imread(f'feather-in-focus/{file_path}{img_path}')
        img = cv2.resize(img, (300, 300))  # Resize the image

        # Normalize pixel values using EfficientNetV2's preprocess_input
        img = efficientnet_v2.preprocess_input(img)

        images.append(img)
    return np.array(images)

In [29]:
# Load and preprocess images
X_images = load_and_preprocess_images(data['image_path'], 'train_images')
y_labels = to_categorical(data['label'].values-1)

X_train, X_test, y_train, y_test = train_test_split(X_images, y_labels, test_size=0.2, random_state=0)

## Using data augmentation to create more data

In [30]:
# # Create an ImageDataGenerator with augmentation parameters, not used currently
# datagen = ImageDataGenerator(
#     rotation_range=20,
#     width_shift_range=0.2,
#     height_shift_range=0.2,
#     shear_range=0.2,
#     zoom_range=0.2,
#     horizontal_flip=True,
#     fill_mode='nearest'
# )

# # Apply data augmentation to your training data
# augmented_data = datagen.flow(np.array(X_train), y_train, batch_size=32)

## Building the model


In [31]:
# Create Xception model with pre-trained weights
base_model =  efficientnet_v2.EfficientNetV2B3(include_top=False, weights='imagenet', input_shape=(300, 300, 3), classes=200)
# Freeze the layers of the pre-trained model
for layer in base_model.layers:
    layer.trainable = False

# Add custom top layers for classification
model = models.Sequential()
model.add(base_model)
model.add(layers.GlobalAveragePooling2D())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.5)) #0.5 works best
model.add(layers.Dense(y_train.shape[1], activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


# allow model to quit when not seeing improvement
early_stop = tf.keras.callbacks.EarlyStopping(
    patience=3,
    min_delta=0.001,
    restore_best_weights=True,)


# Train the model
epochs = 20  
model.fit(np.array(X_train), y_train, epochs=epochs, validation_data=(np.array(X_test), y_test), callbacks=[early_stop])
# model.fit(augmented_data, epochs=epochs, validation_data=(np.array(X_test), y_test)) <- use this if you want to use the augmented data, didnt do much when used


# Evaluate the model
test_loss, test_acc = model.evaluate(np.array(X_test), y_test)
print(f'Test accuracy: {test_acc}')

Epoch 1/20
99/99 [==============================] - 85s 784ms/step - loss: 4.6527 - accuracy: 0.0809 - val_loss: 3.6629 - val_accuracy: 0.2176
Epoch 2/20
99/99 [==============================] - 75s 763ms/step - loss: 3.3717 - accuracy: 0.2242 - val_loss: 2.8920 - val_accuracy: 0.3244
Epoch 3/20
99/99 [==============================] - 72s 732ms/step - loss: 2.7155 - accuracy: 0.3261 - val_loss: 2.4569 - val_accuracy: 0.4033
Epoch 4/20
99/99 [==============================] - 75s 757ms/step - loss: 2.3316 - accuracy: 0.3962 - val_loss: 2.2245 - val_accuracy: 0.4453
Epoch 5/20
99/99 [==============================] - 74s 751ms/step - loss: 2.0750 - accuracy: 0.4373 - val_loss: 2.0362 - val_accuracy: 0.4593
Epoch 6/20
99/99 [==============================] - 74s 750ms/step - loss: 1.8316 - accuracy: 0.4997 - val_loss: 1.9306 - val_accuracy: 0.4606
Epoch 7/20
99/99 [==============================] - 72s 730ms/step - loss: 1.7020 - accuracy: 0.5360 - val_loss: 1.8894 - val_accuracy: 0.4784

In [32]:
# Unfreeze a few layers at a time
for layer in base_model.layers[-20:]:
    layer.trainable = True

# Recompile the model after unfreezing layers
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Continue training
model.fit(np.array(X_train), y_train, epochs=5, validation_data=(np.array(X_test), y_test)) 


Epoch 1/5
99/99 [==============================] - 89s 815ms/step - loss: 1.0437 - accuracy: 0.6869 - val_loss: 1.5878 - val_accuracy: 0.5611
Epoch 2/5
99/99 [==============================] - 78s 790ms/step - loss: 0.8542 - accuracy: 0.7490 - val_loss: 1.6162 - val_accuracy: 0.5471
Epoch 3/5
45/99 [============>.................] - ETA: 35s - loss: 0.6869 - accuracy: 0.7840

## predicting the submission images and saving them 


In [ ]:

test_df = pd.read_csv("feather-in-focus/test_images_path.csv")
test_images = load_and_preprocess_images(test_df['image_path'],'test_images')

# Predict
predictions = model.predict(test_images)
predicted_labels = np.argmax(predictions, axis=1)


submission_df = pd.DataFrame({
    'id': test_df['id'],
    'label': predicted_labels+1
})
submission_df.to_csv('submission.csv', index=False)

125/125 [==============================] - 29s 208ms/step
